In [1]:
import win32com.client
import pandas as pd
import numpy as np
import csv
import glob
import os

In [5]:
import platform

In [6]:
print(platform.architecture())

('32bit', 'WindowsPE')


In [10]:
import sys
print(sys.version)

3.7.7 (default, May  6 2020, 11:43:10) [MSC v.1916 32 bit (Intel)]


In [13]:
import win32com.client
instCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
print(instCpCybos.IsConnect)

0
